In [1]:
import pandas as pd
import numpy as np
from math import pi
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, layout, gridplot
from pandas.tseries.offsets import Hour
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Read S&P 500 quotes 

In [2]:
sp500 = pd.read_csv('../monthly_seasonal/sp500_historical.csv.gz', sep=',')
sp500['Date'] = pd.to_datetime(sp500['Date'])
sp500.set_index('Date', inplace=True)
sp500.tail(10)

,open,high,low,close,volume,adj_close
Date,,,,,,
2017-08-16,2468.629883,2474.929932,2463.860107,2468.110107,2468.110107,2.953650e+09
2017-08-17,2462.949951,2465.020020,2430.010010,2430.010010,2430.010010,3.142620e+09
2017-08-18,2427.639893,2440.270020,2420.689941,2425.550049,2425.550049,3.415680e+09
2017-08-21,2425.500000,2430.580078,2417.350098,2428.370117,2428.370117,2.788150e+09
2017-08-22,2433.750000,2454.770020,2433.669922,2452.510010,2452.510010,2.777490e+09
2017-08-23,2444.879883,2448.909912,2441.419922,2444.040039,2444.040039,2.785290e+09
2017-08-24,2447.909912,2450.389893,2436.189941,2438.969971,2438.969971,2.846590e+09
2017-08-25,2444.719971,2453.959961,2442.219971,2443.050049,2443.050049,2.588780e+09
2017-08-28,2447.350098,2449.120117,2439.030029,2444.239990,2444.239990,2.677700e+09


## Create simple moving average

In [3]:
sp500["SMA_20"] = sp500["close"].rolling(window=20).mean()

## Display candlestick and SMA with Bokeh

In [12]:
bars_disp = 70
df = sp500[-bars_disp:].copy()

# Find up and down bars
inc = df.close > df.open
dec = df.open > df.close

p1 = figure(x_axis_type="datetime", 
            tools="pan,wheel_zoom,box_zoom,reset,save", 
            plot_width=900, plot_height=400, 
            title = "S&P 500 last 70 days")
p1.xaxis.major_label_orientation = pi/4
p1.grid.grid_line_alpha=0.3

# Create candlesticks
p1.segment(df.index, df.high, df.index, df.low, color="black", line_width=2)
w = 12*60*60*1000 # half day in ms
p1.vbar(df.index[inc], w, df.open[inc], df.close[inc], 
        fill_color="#D5E1DD", line_color="black")
p1.vbar(df.index[dec], w, df.open[dec], df.close[dec], 
        fill_color="#F2583E", line_color="black")

# Display SMA
p1.line(df.index, df["SMA_20"], color="red", legend='SMA 20', line_width=2)
p1.legend.location = "bottom_right"

show(p1)

## Display a bar plot with SMA using Bokeh

In [20]:
bars_disp = 70
df = sp500[-bars_disp:].copy()

# Find up and down bars
inc = df.close > df.open
dec = df.open > df.close

w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p1 = figure(x_axis_type="datetime", 
            tools=TOOLS, 
            plot_width=900, plot_height=400, 
            title = "S&P 500 last 70 days")
p1.xaxis.major_label_orientation = pi/4
p1.grid.grid_line_alpha=0.3

# Plot the central high to low vertical line
p1.segment(df.index[inc], df.high[inc], df.index[inc], df.low[inc], color="green", line_width=2)
p1.segment(df.index[dec], df.high[dec], df.index[dec], df.low[dec], color="red", line_width=2)

# Plot horizontal opening line
p1.segment(df.index[inc] - pd.Timedelta(Hour(12)), df.open[inc], 
           df.index[inc], df.open[inc], color="green")
p1.segment(df.index[dec] - pd.Timedelta(Hour(12)), df.open[dec], 
           df.index[dec], df.open[dec], color="red")
# Plot horizontal closing line 
p1.segment(df.index[inc] + pd.Timedelta(Hour(12)), df.close[inc], 
           df.index[inc], df.close[inc], color="green")
p1.segment(df.index[dec] + pd.Timedelta(Hour(12)), df.close[dec], 
           df.index[dec], df.close[dec], color="red")

# Display SMA
p1.line(df.index, df["SMA_20"], color="black", legend='SMA 20', line_width=2)
p1.legend.location = "bottom_right"

show(p1)